In [38]:
from torchaudio.datasets import LJSPEECH
from IPython.display import Audio

dataset = LJSPEECH(root="data", download=True)
waveform, sample_rate, transcript, normalized = dataset[4]

print(transcript)
wav_ref = waveform.squeeze().numpy()
Audio(wav_ref, rate=sample_rate)

the invention of movable metal letters in the middle of the fifteenth century may justly be considered as the invention of the art of printing.


In [ ]:
import torch
from transformers import CsmForConditionalGeneration, AutoProcessor

model_id = "sesame/csm-1b"
device = "cuda" if torch.cuda.is_available() else "cpu"

processor = AutoProcessor.from_pretrained(model_id)
model = CsmForConditionalGeneration.from_pretrained(model_id, device_map=device)

In [42]:
import os, random

save_dir = "tts_out"
os.makedirs(save_dir, exist_ok=True)

random.seed(42)

N = len(dataset)
pick = random.sample(range(N), k=min(20, N))

4506


In [ ]:
for i in pick:
    waveform = dataset[i]

In [45]:
for i in pick:
    trnascript = dataset[i]
    text = transcript
    
    conversation = [
        {
            "role": "0", 
            "content": [{"type": "text", "text": text}],
        },
    ]

    inputs = processor.apply_chat_template(
        conversation,
        tokenize=True,
        return_dict=True,
    ).to(device)
    
    audio = model.generate(**inputs, output_audio=True)
    out_path = os.path.join(save_dir, f"sample_{i:05d}.wav")
    processor.save_audio(audio, out_path)

In [50]:
sample_rate = 25000
#wav_np = (audio[0] if isinstance(audio, (list, tuple)) else audio).detach().cpu().numpy()
files = [f for f in os.listdir(save_dir) if f.endswith(".wav")]
wav_path = os.path.join(save_dir, files[6])
Audio(wav_path)